# scrapy 못써먹겠다

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

## 1. 기사목록
- 날짜, 대분류, 소분류, 페이지

In [8]:
date_list = list(range(20230201, 20230229)) + list(range(20230301, 20230317))
#대분류
main_categories = {'101' : '경제', '105' : 'IT/과학'}
#소분류
sub_categories = {
    # #경제
    # '259' : '금융',
    # '258' : '증권',
    # '260' : '부동산',
    # '261' : '산업/제계',
    # '262' : '글로벌 경제',
    # '263' : '경제일반',
    # '310' : '생활경제',
    # '771' : '중기/벤처',

    #IT/과학
    '226' : '인터넷/SNS',
    '227' : '통신/뉴미디어',
    '228' : '과학/일반',
    '229' : '게임/리뷰',
    '230' : 'IT일반',
    '283' : '컴퓨터',
    '731' : '모바일',
    '732' : '보안/해킹'
    }

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
page_urls = []
# for date in date_list:
#     for main_category in main_categories:
#         for sub_category in sub_categories:
#             page_n = 1
#             while True:
#                 page_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2={sub_category}&sid1={main_category}&date={date}&page={page_n}'
            
#                 r = requests.get(page_url, headers=headers)
#                 r_soup = BeautifulSoup(r.text, "html.parser")
                
#                 r2 = requests.get(f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2={sub_category}&sid1={main_category}&date={date}&page={page_n +1}', headers=headers)
#                 r2_soup = BeautifulSoup(r2.text, "html.parser")
                
#                 if r_soup == r2_soup:
#                     break
#                 else:
#                     page_urls.append(page_url)
                
#                 page_n += 1


for date in date_list:
    for sub_category in sub_categories:
        page_n = 1
        while True:
            page_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2={sub_category}&sid1=105&date={date}&page={page_n}'
        
            r = requests.get(page_url, headers=headers)
            r_soup = BeautifulSoup(r.text, "html.parser")
            
            r2 = requests.get(page_url[:94]+str(page_n+1), headers=headers)
            r2_soup = BeautifulSoup(r2.text, "html.parser")
            
            if r_soup == r2_soup:
                break
            else:
                page_urls.append(page_url)
            
            page_n += 1
            break





page_urls


['https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=226&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=227&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=228&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=229&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=230&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=283&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=731&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=732&sid1=105&date=20230201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=226&sid1=105&date=20230202&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=227&sid1=105&date=20230202&page=1',


In [6]:
len('https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=105&sid1=105&date=20230304&page=')


94

## 2. 기사 별 크롤링

In [16]:
datasheet = pd.DataFrame(columns=['ID', 'platform', 'main_category', 'sub_category', 'title', 'content', 'writer', 'writed_at'])



for page_url in page_urls:  #페이지 내 기사별 url 수집 - 기사 별 크롤링 준비
    URL = page_url
    r = requests.get(URL, headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'})
    soup = BeautifulSoup(r.text, "html.parser")
    result = list(set(soup.select('li dt a')))
    new_result = [i.attrs['href'] for i in result]

    #수집한 기사 링크를 바탕으로 필요한 정보 parsing
    


['https://n.news.naver.com/mnews/article/018/0005416645?sid=101',
 'https://n.news.naver.com/mnews/article/014/0004963203?sid=101',
 'https://n.news.naver.com/mnews/article/119/0002679972?sid=101',
 'https://n.news.naver.com/mnews/article/014/0004963206?sid=101',
 'https://n.news.naver.com/mnews/article/003/0011670499?sid=101',
 'https://n.news.naver.com/mnews/article/014/0004963213?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005416645?sid=101',
 'https://n.news.naver.com/mnews/article/016/0002098040?sid=101',
 'https://n.news.naver.com/mnews/article/016/0002098040?sid=101',
 'https://n.news.naver.com/mnews/article/009/0005082498?sid=101',
 'https://n.news.naver.com/mnews/article/119/0002679972?sid=101',
 'https://n.news.naver.com/mnews/article/029/0002781770?sid=101',
 'https://n.news.naver.com/mnews/article/003/0011670499?sid=101',
 'https://n.news.naver.com/mnews/article/031/0000725947?sid=101',
 'https://n.news.naver.com/mnews/article/016/0002098030?sid=101',
 'https://